# Export Data via API for Import to Database

This exports all the major data from Lightspeed POS to spreadsheets. This is helpful if you need to migrate off lightspeed, or you just want backups of all the major things

In [6]:
import logging
import requests
import json
import pandas as pd

# Start logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')


2020-11-19 19:28:59,437 - DEBUG - Start of program


## Import the Module
Make sure your environment paths are set correctly if you are getting module not found errors.

In [7]:
from pyLightspeed.lsretail import api
from pyLightspeed.lsecom import api as lsecom

## Get Keys
This example uses a file you will have to create on your local machine to store keys. This is not best practice - you should probably store your keys in environment variables - but is simpler for people just trying to test and play. See the example files in example\data and adjust with your information.

In [8]:
KEY_FILE = "D:\Development\.keys\lightspeed_keys.json"
CODES_FILE = "vintage_codes.json"



with open(KEY_FILE) as f:
    keys = json.load(f)

store_data = {
            'account_id': keys["account_id"],
            'save_path': 'D:\\Development\\.keys\\'
            }

credentials = {
            'client_id': keys["client_id"],
            'client_secret': keys["client_secret"],
            'api_key': keys["api_key"],
            'api_secret': keys["api_secret"]
            }

## Create a Connection
Call the connection class, which will return an active connection to Lightspeed.

In [9]:
def write_out(filepath, resource):
    to_json = open(filepath + '.json', 'w')
    to_json.write(json.dumps(resource, indent=4))
    to_json.close()
    pd.DataFrame(resource).to_csv(filepath +'.csv', index=False, encoding='utf-8-sig')
    logging.debug(f'Wrote data to {filepath}')

In [10]:
# Creates the connection to lightspeed, and returns a connection object with useful properties
lsr = api.Connection(store_data, credentials, codes_file = CODES_FILE)

lse = lsecom.Connection(store_data, credentials, codes_file = CODES_FILE)


TypeError: __init__() missing 1 required positional argument: 'codes_file'

## Export things

This takes a tuple of API endpoints (make sure they are spelled right) and loops through them, dumping the results to a .csn and raw JSON file

In [ ]:
#Get a list of things

exports = ('Sale', 'SaleLine','CustomerType','Manufacturer','Vendor', 'Order', 'OrderLine')
#These are updated rarely, so likely don't need to run them, but here if you want to refresh
#exports = ('Category','Employee', 'Register','CustomerType','TaxCategory' )


for export in exports:
    resource = lsr.list(export, filter='archived=true')
    write_out(lse.save_path + export, resource)



### Customers are Special
For Customers we want to load with the Contact area

In [ ]:
customers = lsr.list("Customer", filter = 'load_relations=["Contact"]&archived=true')
write_out(lse.save_path + 'Customer_Contact', customers)

In [ ]:
# Items are Special
# We need to get some Quantity on Hand information, so we need to include the relationship.
items = lsr.list("Item", filter = 'load_relations=["ItemShops"]&archived=true')
write_out(lse.save_path + 'Item_Shop', items)


# Export eCom transactions

In [ ]:
#Get a list of things

#exports = ('orders', 'customers', 'products')
exports = ('orders', 'customers', 'products', 'filters', 'tags','discounts')
#exports = ('orders', 'customers', 'products')


for export in exports:
    resource = lse.list(export)
    write_out(lse.save_path + 'ecom_' + export, resource)
 


In [ ]:
# orders = lse.list('orders')
# order_products = []
# write_out(lse.save_path+'/'+ 'ecom_' + export, orders)

# for order in orders:
#     resource = lse.list(order['products']['resource']['url'])
#     all_data = lse.response.json()
#     order_products.extend(all_data)
    
    


#     while total_amount > current_offset:
        
#         querystring = {'offset':current_offset, 'limit':current_limit}
#         self.response = requests.get(url, params=querystring, headers=self.headers)
#         self.response.raise_for_status()
#         all_data = self.response.json()
#         all_resources.extend(all_data[resource])
#         current_offset = current_offset + current_limit